In [27]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id
spark = SparkSession.builder.getOrCreate()

data = spark.read.text("/Users/zyfei/Desktop/pima_indian.txt").rdd
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),
                                     rating=float(p[2]), timestamp=long(p[3])))
#data = data.withColumn("id", monotonically_increasing_id())
data.printSchema()

root
 |-- Pregnant_time: integer (nullable = true)
 |-- Plasma_glucose_concentration: integer (nullable = true)
 |-- Blood_pressure: integer (nullable = true)
 |-- Skin_fold_thickness: integer (nullable = true)
 |-- Serum_insulin: integer (nullable = true)
 |-- Body_mass_index: double (nullable = true)
 |-- Diabetes_pedigree: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- label: integer (nullable = true)



In [28]:
data.filter(data.label == 1).count()
#data.filter(data.Label == 0).count()

268

In [29]:
from pyspark.ml.feature import VectorAssembler
label = ["label"]
assembler = VectorAssembler(
    inputCols=[x for x in data.columns if x not in label],
    outputCol='features')
data = assembler.transform(data)

In [30]:
#data.select('features').show(10,False)
data = data.select('label','features')
data.printSchema()

root
 |-- label: integer (nullable = true)
 |-- features: vector (nullable = true)



In [31]:

# Split the data into train and test
splits = data.select("label", "features").randomSplit([0.8, 0.2], 1234)
train = splits[1]
test = splits[0]



In [32]:
train.filter(train.label == 1).show(10,False)

+-----+-------------------------------------------+
|label|features                                   |
+-----+-------------------------------------------+
|1    |(8,[1,5,6,7],[119.0,32.4,0.141,24.0])      |
|1    |(8,[1,5,6,7],[138.0,36.3,0.933,25.0])      |
|1    |(8,[1,5,6,7],[145.0,44.2,0.63,31.0])       |
|1    |[0.0,104.0,64.0,37.0,64.0,33.6,0.51,22.0]  |
|1    |[0.0,131.0,88.0,0.0,0.0,31.6,0.743,32.0]   |
|1    |[0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0]|
|1    |[0.0,179.0,50.0,36.0,159.0,37.8,0.455,22.0]|
|1    |[0.0,189.0,104.0,25.0,0.0,34.3,0.435,41.0] |
|1    |[1.0,149.0,68.0,29.0,127.0,29.3,0.349,42.0]|
|1    |[1.0,172.0,68.0,49.0,579.0,42.4,0.702,28.0]|
+-----+-------------------------------------------+
only showing top 10 rows



In [38]:
#NaiveBayes
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

nb = NaiveBayes()

# train the model
model = nb.fit(train)

# select example rows to display.
predictions = model.transform(test)
#predictions.show(10)

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.61648177496


In [37]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

dt = DecisionTreeClassifier()

model = dt.fit(train)

predictions = model.transform(test)

evaluator = MulticlassClassificationEvaluator(labelCol="label",
                                             predictionCol="prediction",
                                             metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.698890649762


In [39]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

rf = RandomForestClassifier()

model = rf.fit(train)

predictions = model.transform(test)

evaluator = MulticlassClassificationEvaluator(labelCol="label",
                                             predictionCol="prediction",
                                             metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy of RandomForest= " + str(accuracy))

Test set accuracy = 0.765451664025


In [40]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

ls = LinearSVC()

model = ls.fit(train)

predictions = model.transform(test)

evaluator = MulticlassClassificationEvaluator(labelCol="label",
                                             predictionCol="prediction",
                                             metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy of LinearSVC= " + str(accuracy))

Test set accuracy = 0.75911251981
